<a href="https://colab.research.google.com/github/wizard339/education/blob/main/misis/nlp/token_classification/token_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate seqeval

In [36]:
import pandas as pd
import numpy as np
import evaluate
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer
from sklearn.preprocessing import LabelEncoder

In [3]:
!wget 'https://raw.githubusercontent.com/wizard339/education/main/misis/nlp/token_classification/train.txt'
!wget 'https://raw.githubusercontent.com/wizard339/education/main/misis/nlp/token_classification/dev.txt'
!wget 'https://raw.githubusercontent.com/wizard339/education/main/misis/nlp/token_classification/test.txt'

--2023-03-15 20:13:34--  https://raw.githubusercontent.com/wizard339/education/main/misis/nlp/token_classification/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2302972 (2.2M) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>]   2.20M  --.-KB/s    in 0.07s   

2023-03-15 20:13:34 (29.4 MB/s) - ‘train.txt’ saved [2302972/2302972]

--2023-03-15 20:13:35--  https://raw.githubusercontent.com/wizard339/education/main/misis/nlp/token_classification/dev.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200

In [4]:
with open('/content/train.txt', 'r') as f:
    train_txt = f.read()

with open('/content/dev.txt', 'r') as f:
    val_txt = f.read()

with open('/content/test.txt', 'r') as f:
    test_txt = f.read()

In [5]:
def text_preproc(text: str) -> pd.DataFrame:
    '''
    preprocessing input texts 
    '''
    splitted_text = text.split('\n')
    sentences, tags = [], []
    sentence, tag = [], []
    
    for text in splitted_text:

        if text != '':
            text = text.split(' ')
            sentence.append(text[0])
            tag.append(text[1])
        else:
            sentences.append(sentence)
            tags.append(tag)
            sentence, tag = [], []
    
    df = pd.DataFrame({'Text': sentences,
                       'Labels': tags})

    return df

In [6]:
train_data = text_preproc(train_txt)
print(train_data.shape)
train_data.head()

(7747, 2)


,Text,Labels
0,"["", Если, Миронов, занял, столь, оппозиционную...","[O, O, B-PER, O, O, O, O, O, O, O, O, O, O, O,..."
1,"[Источник, "", Ъ, '', в, руководстве, столичной...","[O, O, B-ORG, O, O, O, O, O, O, O, O, O, O, B-..."
2,"[В, Ханты-Мансийском, автономном, округе, с, д...","[O, B-LOC, I-LOC, I-LOC, O, O, O, O, B-ORG, B-..."
3,"[С, 1992, года, по, настоящее, время, является...","[O, O, O, O, O, O, O, O, B-ORG, I-ORG, I-ORG, ..."
4,"[Для, этого, ей, пришлось, выиграть, выборы, в...","[O, O, O, O, O, O, O, O, O, O, B-LOC, I-LOC, O..."


In [7]:
val_data = text_preproc(val_txt)
print(val_data.shape)
val_data.head()

(2583, 2)


,Text,Labels
0,"[как, акционерный, коммерческий, Московский, м...","[O, O, O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I..."
1,"[Управлять, ЦАО, и, САО, вместо, Алексея, Алек...","[O, B-LOC, O, B-LOC, O, B-PER, I-PER, O, B-PER..."
2,"[О, задержании, Шакирьянова, стало, известно, ...","[O, O, B-PER, O, O, O, O, O, O, O]"
3,"[После, майского, ухода, вице-премьера, Владис...","[O, O, O, O, B-PER, I-PER, O, O, O, O, O, B-PE..."
4,"[Армяне, со, мной, согласились, ,, с, Ильхамом...","[O, O, O, O, O, O, B-PER, I-PER, O, O, O, O, O..."


In [8]:
test_data = text_preproc(test_txt)
print(test_data.shape)
test_data.head()

(2583, 2)


,Text,Labels
0,"[Тогда, замешанные, в, скандале, прокуроры, от...","[O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,"[Символичным, назвал, председатель, РФМ, ,, де...","[O, O, O, B-ORG, O, O, B-ORG, I-ORG, B-LOC, B-..."
2,"[На, посту, гендиректора, Yahoo, !, B-ORG, Кэр...","[O, O, O, O, O, O, B-PER, I-PER, O, B-PER, I-P..."
3,"[Считаю, невозможным, руководить, областью, с,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[Боксер, победила, Карли, Фиорину, (, Carly, F...","[B-PER, O, B-PER, I-PER, O, I-PER, O, O, O, O,..."


In [9]:
ner_tags = []
for labels_list in train_data['Labels']:
    for label in labels_list:
        ner_tags.append(label)
ner_tags = list(set(ner_tags))
print(ner_tags)

['B-LOC', 'O', 'I-ORG', 'I-PER', 'B-ORG', 'I-LOC', 'B-PER']


In [10]:
ner_tags = dict(zip(['O', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER'], list(range(7))))
print(ner_tags)

{'O': 0, 'B-LOC': 1, 'I-LOC': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-PER': 5, 'I-PER': 6}


In [11]:
ner_tags_inv = {v: k for k, v in ner_tags.items()}
print(ner_tags_inv)

{0: 'O', 1: 'B-LOC', 2: 'I-LOC', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-PER', 6: 'I-PER'}


In [12]:
def label_encode(labels):
    new_labels = []
    for label in labels:
        new_labels.append(ner_tags[label])
    return new_labels

train_data['Labels'] = train_data['Labels'].apply(label_encode)
val_data['Labels'] = val_data['Labels'].apply(label_encode)
test_data['Labels'] = test_data['Labels'].apply(label_encode)

In [13]:
train_data.head()

,Text,Labels
0,"["", Если, Миронов, занял, столь, оппозиционную...","[0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[Источник, "", Ъ, '', в, руководстве, столичной...","[0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, ..."
2,"[В, Ханты-Мансийском, автономном, округе, с, д...","[0, 1, 2, 2, 0, 0, 0, 0, 3, 5, 6, 0]"
3,"[С, 1992, года, по, настоящее, время, является...","[0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 0, 4, 0, 0, ..."
4,"[Для, этого, ей, пришлось, выиграть, выборы, в...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]"


In [14]:
hg_train = Dataset.from_pandas(train_data)
assert hg_train.shape == train_data.shape
print(f'Train shape: {hg_train.shape}')

hg_val = Dataset.from_pandas(val_data)
assert hg_val.shape == val_data.shape
print(f'Valid shape: {hg_val.shape}')

hg_test = Dataset.from_pandas(test_data)
assert hg_test.shape == test_data.shape
print(f'Test shape: {hg_test.shape}')

print(hg_train)
print(hg_val)
print(hg_test)

Train shape: (7747, 2)
Valid shape: (2583, 2)
Test shape: (2583, 2)
Dataset({
    features: ['Text', 'Labels'],
    num_rows: 7747
})
Dataset({
    features: ['Text', 'Labels'],
    num_rows: 2583
})
Dataset({
    features: ['Text', 'Labels'],
    num_rows: 2583
})


In [17]:
model_checkpoint = 'xlm-roberta-large-finetuned-conll03-english'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [18]:
tokenizer.is_fast

True

In [19]:
inputs = tokenizer(hg_train['Text'][0], is_split_into_words=True)
print(inputs.tokens())

['<s>', '▁"', '▁Если', '▁Мир', 'о', 'нов', '▁за', 'нял', '▁столь', '▁оп', 'пози', 'цион', 'ную', '▁позицию', '▁', ',', '▁то', '▁мне', '▁представляет', 'ся', '▁', ',', '▁что', '▁для', '▁него', '▁было', '▁бы', '▁поряд', 'о', 'чным', '▁и', '▁прави', 'льным', '▁уйти', '▁в', '▁от', 'ставку', '▁с', '▁занима', 'емого', '▁им', '▁поста', '▁', ',', '▁поста', '▁', ',', '▁который', '▁предо', 'ставлен', '▁ему', '▁сегодня', '▁"', '▁Един', 'ой', '▁Россией', "▁''", '▁и', '▁ни', 'кем', '▁больше', "▁''", '▁', ',', '▁-', '▁заключа', 'ет', '▁Иса', 'ев', '▁', '.', '</s>']


In [20]:
print(inputs.word_ids())

[None, 0, 1, 2, 2, 2, 3, 3, 4, 5, 5, 5, 5, 6, 7, 7, 8, 9, 10, 10, 11, 11, 12, 13, 14, 15, 16, 17, 17, 17, 18, 19, 19, 20, 21, 22, 22, 23, 24, 24, 25, 26, 27, 27, 28, 29, 29, 30, 31, 31, 32, 33, 34, 35, 35, 36, 37, 38, 39, 39, 40, 41, 42, 42, 43, 44, 44, 45, 45, 46, 46, None]


In [21]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as precious token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [22]:
labels = hg_train['Labels'][0]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0]
[-100, 0, 0, 5, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0, -100]


In [23]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['Text'], truncation=True, is_split_into_words=True
    )
    all_labels = examples['Labels']
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs['labels'] = new_labels
    return tokenized_inputs

In [24]:
tokenized_train = hg_train.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=hg_train.column_names
)

tokenized_val = hg_val.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=hg_val.column_names
)

tokenized_test = hg_test.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=hg_test.column_names
)

print(tokenized_train)
print(tokenized_val)
print(tokenized_test)

Map:   0%|          | 0/7747 [00:00<?, ? examples/s]

Map:   0%|          | 0/2583 [00:00<?, ? examples/s]

Map:   0%|          | 0/2583 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 7747
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2583
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2583
})


In [25]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [26]:
batch = data_collator([tokenized_train[i] for i in range(2)])
batch['labels']

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    0,    0,    5,    6,    6,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    3,    4,    4,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    5,    6,    0,    0, -100],
        [-100,    0,    0,    3,    4,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    5,    6,    6,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    1,    2,    2,    2,    2,    0,    0,    0, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

In [27]:
metric = evaluate.load('seqeval')

In [28]:
labels = train_data['Labels'][0]
labels = [ner_tags_inv[i] for i in labels]
print(labels)

['O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O']


In [29]:
fake_predictions = labels.copy()
fake_predictions[2] = 'O'
metric.compute(predictions=[fake_predictions], references=[labels])

{'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'PER': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.9787234042553191}

In [30]:
def compute_metric(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[ner_tags_inv[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [ner_tags_inv[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        'precision': all_metrics['overall_precision'],
        'recall'   : all_metrics['overall_recall'],
        'f1'       : all_metrics['overall_f1'],
        'accuracy' : all_metrics['overall_accuracy'],
    }

In [31]:
id2label = ner_tags
label2id = ner_tags_inv

In [33]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large-finetuned-conll03-english and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([8, 1024]) in the checkpoint and torch.Size([7, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([8]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
args = TrainingArguments(
    model_checkpoint,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    compute_metrics=compute_metric,
    tokenizer=tokenizer,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
